In [231]:
import numpy as np
import pandas as pd
import os
import itertools as it
import time

In [232]:
pd.options.display.max_rows = 500

In [233]:
# file = ''
# for i in os.listdir():
#     if 'NBA' in i:
#         file = i

In [234]:
raw = pd.read_csv('FanDuel-MLB-2019-04-08-34316-players-list.csv')

In [235]:
raw.columns
raw.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Probable Pitcher,Batting Order
0,34316-5481,P,Max,Max Scherzer,Scherzer,48.0,2,11900,WAS@PHI,WAS,PHI,NaN,NaN,NaN,NaN,NaN
1,34316-16956,P,Gerrit,Gerrit Cole,Cole,44.5,2,11200,NYY@HOU,HOU,NYY,NaN,NaN,NaN,NaN,NaN
2,34316-5202,P,Justin,Justin Verlander,Verlander,33.5,2,10700,NYY@HOU,HOU,NYY,NaN,NaN,NaN,Yes,NaN
3,34316-60647,P,Aaron,Aaron Nola,Nola,23.0,2,10700,WAS@PHI,PHI,WAS,NaN,NaN,NaN,NaN,NaN
4,34316-21098,P,James,James Paxton,Paxton,33.0,2,10500,NYY@HOU,NYY,HOU,NaN,NaN,NaN,NaN,NaN


In [236]:
p = raw[(raw.Position=='P')&(raw['Probable Pitcher']=='Yes')]

In [237]:
p = p.drop(['Id', 'First Name', 'Last Name', 'Played', 'Game', 'Team', 'Opponent',
               'Injury Details', 'Tier' ,'Probable Pitcher', 'Batting Order'], axis=1)

In [238]:
p.columns = ['position', 'name', 'points', 'salary', 'injury']

In [239]:
p = p[p.injury.isna()]

In [240]:
p.drop('injury', axis=1, inplace=True)

In [241]:
p.shape


(16, 4)

In [242]:
df = raw.drop(['Id', 'First Name', 'Last Name', 'Played', 'Game', 'Team', 'Opponent',
               'Injury Details', 'Tier' ,'Probable Pitcher', 'Batting Order'], axis=1)

In [243]:
df.shape

(492, 5)

In [244]:
df.columns = ['position', 'name', 'points', 'salary', 'injury']
df = df[df.injury.isna()]

In [245]:
df.shape

(414, 5)

In [246]:
df.drop('injury', axis=1, inplace=True)
df.position.unique()

array(['P', 'OF', '1B', '3B', '2B', 'SS', 'C'], dtype=object)

In [247]:
# p = df[df.position == 'P']
of = df[df.position == 'OF']
b3 = df[df.position == '3B']
b2 = df[df.position == '2B']
b1c = df[(df.position == '1B') | (df.position=='C')]
ss = df[df.position == 'SS']

In [248]:
('p', p.shape[0]), ('of', of.shape[0]), ('3b', b3.shape[0]), ('2b', b2.shape[0]), ('b1c', b1c.shape[0]), ('ss', ss.shape[0])

(('p', 16), ('of', 75), ('3b', 24), ('2b', 23), ('b1c', 54), ('ss', 24))

### Bases

In [249]:
b2_3 = b2.assign(foo=1).merge(b3.assign(foo=1), on='foo').drop('foo', 1)

In [250]:
# x-2b, y-3b
# b2_3['salary'] = 

b2_3['salary'] = b2_3.salary_x + b2_3.salary_y
b2_3['points'] = b2_3.points_x + b2_3.points_y

In [251]:
b2_3.drop(['position_x', 'position_y', 'points_x', 'points_y', 'salary_x', 'salary_y'], axis=1, inplace=True)

In [252]:
b2_3.columns = ['2b', '3b', 'salary', 'points']

In [253]:
b2_3 = b2_3.sort_values(by='points', ascending=False)

##### 2nd/3rd + 1st/catcher

In [254]:
b123c = b2_3.assign(foo=1).merge(b1c.assign(foo=1), on='foo').drop('foo', 1)

In [255]:
b123c.head()

,2b,3b,salary_x,points_x,position,name,points_y,salary_y
0,Kolten Wong,Anthony Rendon,8100,37.405357,1B,Freddie Freeman,12.687500,4900
1,Kolten Wong,Anthony Rendon,8100,37.405357,1B,Rhys Hoskins,18.557142,4600
2,Kolten Wong,Anthony Rendon,8100,37.405357,1B,Paul Goldschmidt,16.437500,4200
3,Kolten Wong,Anthony Rendon,8100,37.405357,1B,Max Muncy,11.825000,3900
4,Kolten Wong,Anthony Rendon,8100,37.405357,1B,Luke Voit,11.825000,3700


In [256]:
b123c['salary'] = b123c.salary_x + b123c.salary_y
b123c['points'] = b123c.points_x + b123c.points_y

In [257]:
b123c.drop(['salary_x', 'points_x', 'position', 'points_y', 'salary_y'], axis=1, inplace=True)

In [258]:
b123c = b123c.rename(columns={'name':'1bc'})

In [259]:
b123c.shape

(29808, 5)

In [260]:
b123c.sort_values('points', ascending=False)

,2b,3b,1bc,salary,points
1,Kolten Wong,Anthony Rendon,Rhys Hoskins,12700,55.962499
55,Kolten Wong,Maikel Franco,Rhys Hoskins,11900,54.848213
2,Kolten Wong,Anthony Rendon,Paul Goldschmidt,12300,53.842857
109,Kike Hernandez,Anthony Rendon,Rhys Hoskins,12800,53.674999
56,Kolten Wong,Maikel Franco,Paul Goldschmidt,11500,52.728571
163,Kike Hernandez,Maikel Franco,Rhys Hoskins,12000,52.560713
217,Kolten Wong,Ryon Healy,Rhys Hoskins,12000,52.449642
271,Whit Merrifield,Anthony Rendon,Rhys Hoskins,13400,52.328571
325,Jonathan Villar,Anthony Rendon,Rhys Hoskins,12900,51.624999
110,Kike Hernandez,Anthony Rendon,Paul Goldschmidt,12400,51.555357


### Infield - b123c w/ ss

In [261]:
inf = b123c.assign(foo=1).merge(ss.assign(foo=1), on='foo').drop('foo', 1)

In [262]:
inf.head()

,2b,3b,1bc,salary_x,points_x,position,name,points_y,salary_y
0,Kolten Wong,Anthony Rendon,Freddie Freeman,13000,50.092857,SS,Trevor Story,10.388889,4300
1,Kolten Wong,Anthony Rendon,Freddie Freeman,13000,50.092857,SS,Adalberto Mondesi,14.214286,4100
2,Kolten Wong,Anthony Rendon,Freddie Freeman,13000,50.092857,SS,Gleyber Torres,11.600000,4000
3,Kolten Wong,Anthony Rendon,Freddie Freeman,13000,50.092857,SS,Dansby Swanson,12.937500,3900
4,Kolten Wong,Anthony Rendon,Freddie Freeman,13000,50.092857,SS,Tim Beckham,18.770000,3800


In [263]:
inf['salary'] = inf.salary_x + inf.salary_y
inf['points'] = inf.points_x + inf.points_y

In [264]:
inf.drop(['salary_x', 'points_x', 'position', 'points_y', 'salary_y'], axis=1, inplace=True)

In [265]:
inf = inf.rename(columns={'name':'ss'})

In [266]:
inf.head()

,2b,3b,1bc,ss,salary,points
0,Kolten Wong,Anthony Rendon,Freddie Freeman,Trevor Story,17300,60.481746
1,Kolten Wong,Anthony Rendon,Freddie Freeman,Adalberto Mondesi,17100,64.307143
2,Kolten Wong,Anthony Rendon,Freddie Freeman,Gleyber Torres,17000,61.692857
3,Kolten Wong,Anthony Rendon,Freddie Freeman,Dansby Swanson,16900,63.030357
4,Kolten Wong,Anthony Rendon,Freddie Freeman,Tim Beckham,16800,68.862857


In [267]:
inf.columns

Index(['2b', '3b', '1bc', 'ss', 'salary', 'points'], dtype='object')

In [268]:
inf.shape

(715392, 6)

### Outfielders

In [277]:
# three positions w/o replacement

### Pitchers and Outfield

In [269]:
pof = p.assign(foo=1).merge(of.assign(foo=1), on='foo').drop('foo', 1)

In [270]:
pof['salary'] = pof.salary_x + pof.salary_y
pof['points'] = pof.points_x + pof.points_y

In [271]:
pof = pof.rename(columns={'name_x':'p', 'name_y': 'of'})

In [272]:
pof.drop(['position_x', 'points_x', 'salary_x', 'position_y', 'points_y', 'salary_y'], axis=1, inplace=True)

In [273]:
pof.columns

Index(['p', 'of', 'salary', 'points'], dtype='object')

In [274]:
pof.shape

(1200, 4)

In [275]:
pof.head()

,p,of,salary,points
0,Justin Verlander,Mike Trout,15800,51.500000
1,Justin Verlander,Christian Yelich,15700,53.611111
2,Justin Verlander,Ronald Acuna Jr.,15600,44.775000
3,Justin Verlander,Bryce Harper,15500,51.600000
4,Justin Verlander,Cody Bellinger,15400,58.733334


### Full Lines

In [276]:
cols = ['2b', '3b', '1bc', 'ss', 'p', 'of', 'salary', 'points']
df = pd.DataFrame(columns=cols)
t = time.time()
# gg = gs
for i in range(pof.shape[0]):
    ii = inf
    if df.shape[0] >= 100:
        df = df.sort_values(by=['points'], ascending=False)
        df = df.iloc[:100,:]
        ii =  ii[pof.iloc[i,3]+ii.points > df.points.min()]
       
    cap = ii[pof.iloc[i,2]+ii.salary<=60000]
    if cap.shape[0] > 0:
        cap = cap.sort_values(by=['points'], ascending=False)
        cap = cap.iloc[:100,:]
    else:
        continue
    r = pof.iloc[i,:].to_frame().transpose()
    row = pd.DataFrame(r).assign(foo=1).merge(cap.assign(foo=1), on='foo').drop('foo', 1)
    row['points'] = row.points_x + row.points_y
    row['salary'] = row.salary_x + row.salary_y
    row.drop(['points_x', 'points_y', 'salary_x', 'salary_y'], axis=1, inplace=True)
#     row = pd.concat([cf.iloc[i,:5], 
#            cap['sg1':'pg2'], 
#            pd.Series({'salary': cap.salary+cf.iloc[i,6]}),
#             pd.Series({'points': cap.points+cf.iloc[i,5]})])
    df = df.append(row, ignore_index=True)

df

,1bc,2b,3b,of,p,points,salary,ss
0,Rhys Hoskins,Kolten Wong,Anthony Rendon,Cody Bellinger,Kyle Freeland,140.466,29700,Tim Beckham
1,Rhys Hoskins,Kolten Wong,Anthony Rendon,Cody Bellinger,Madison Bumgarner,139.966,30400,Tim Beckham
2,Rhys Hoskins,Kolten Wong,Maikel Franco,Cody Bellinger,Kyle Freeland,139.352,28900,Tim Beckham
3,Rhys Hoskins,Kolten Wong,Maikel Franco,Cody Bellinger,Madison Bumgarner,138.852,29600,Tim Beckham
4,Rhys Hoskins,Kolten Wong,Anthony Rendon,Cody Bellinger,Masahiro Tanaka,138.466,30900,Tim Beckham
5,Paul Goldschmidt,Kolten Wong,Anthony Rendon,Cody Bellinger,Kyle Freeland,138.346,29300,Tim Beckham
6,Rhys Hoskins,Kike Hernandez,Anthony Rendon,Cody Bellinger,Kyle Freeland,138.178,29800,Tim Beckham
7,Paul Goldschmidt,Kolten Wong,Anthony Rendon,Cody Bellinger,Madison Bumgarner,137.846,30000,Tim Beckham
8,Rhys Hoskins,Kike Hernandez,Anthony Rendon,Cody Bellinger,Madison Bumgarner,137.678,30500,Tim Beckham
9,Rhys Hoskins,Kolten Wong,Maikel Franco,Cody Bellinger,Masahiro Tanaka,137.352,30100,Tim Beckham


In [9]:
# def position(df):
#     pos = df.assing(foo=1).merge(df.assign(foo=1), on='foo').drop('foo', 1)
#     pos = pos[pos.Nickname_x != pos.Nickname_y]

### PG's

In [10]:
pgs = pg.assign(foo=1).merge(pg.assign(foo=1), on='foo').drop('foo', 1)
pgs.shape

(529, 12)

In [11]:
pgs = pgs[pgs.Nickname_x != pgs.Nickname_y]

In [12]:
pgs['ffpg'] = pgs.FPPG_x + pgs.FPPG_y
pgs['salary'] = pgs.Salary_x + pgs.Salary_y
pgs_no_opps = pgs[pgs.Team_x != pgs.Opponent_y]
pgs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [13]:
pgs.shape

(506, 4)

### SG's

In [14]:
sg.shape

(32, 6)

In [15]:
sgs = sg.assign(foo=1).merge(sg.assign(foo=1), on='foo').drop('foo', 1)
sgs = sgs[sgs.Nickname_x != sgs.Nickname_y]
sgs['ffpg'] = sgs.FPPG_x + sgs.FPPG_y
sgs['salary'] = sgs.Salary_x + sgs.Salary_y
sgs_no_opps = sgs[sgs.Team_x != sgs.Opponent_y]
sgs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [16]:
sgs.shape

(992, 4)

### SF's

In [17]:
sf.shape

(30, 6)

In [18]:
sfs = sf.assign(foo=1).merge(sf.assign(foo=1), on='foo').drop('foo', 1)
sfs = sfs[sfs.Nickname_x != sfs.Nickname_y]
sfs['ffpg'] = sfs.FPPG_x + sfs.FPPG_y
sfs['salary'] = sfs.Salary_x + sfs.Salary_y
sfs_no_opps = sfs[sfs.Team_x != sfs.Opponent_y]
sfs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [19]:
sfs.shape

(870, 4)

### PF's

In [20]:
pf.shape

(19, 6)

In [21]:
pfs = pf.assign(foo=1).merge(pf.assign(foo=1), on='foo').drop('foo', 1)
pfs = pfs[pfs.Nickname_x != pfs.Nickname_y]
pfs['ffpg'] = pfs.FPPG_x + pfs.FPPG_y
pfs['salary'] = pfs.Salary_x + pfs.Salary_y
pfs_no_opps = pfs[pfs.Team_x != pfs.Opponent_y]
pfs.drop(['Position_x', 'FPPG_x', 'Salary_x', 'Team_x', 'Opponent_x', 'Position_y', 'FPPG_y', 
         'Salary_y', 'Team_y', 'Opponent_y'], axis=1, inplace=True)

In [22]:
# pfs = pfs[(pfs.Nickname_x != 'Anthony Davis') & (pfs.Nickname_y != 'Anthony Davis')]

### Forwards

In [23]:
forwards = pfs.assign(foo=1).merge(sfs.assign(foo=1), on='foo').drop('foo', 1)
forwards['ffpg'] = forwards.ffpg_x + forwards.ffpg_y
forwards['salary'] = forwards.salary_x + forwards.salary_y
forwards.drop(['ffpg_x', 'salary_x', 'ffpg_y', 'salary_y'], axis=1, inplace=True)

In [24]:
forwards.head()

,Nickname_x_x,Nickname_y_x,Nickname_x_y,Nickname_y_y,ffpg,salary
0,LaMarcus Aldridge,Montrezl Harrell,Kevin Durant,Danilo Gallinari,149.840162,34400
1,LaMarcus Aldridge,Montrezl Harrell,Kevin Durant,Jabari Parker,144.757649,33300
2,LaMarcus Aldridge,Montrezl Harrell,Kevin Durant,Rudy Gay,145.773369,33000
3,LaMarcus Aldridge,Montrezl Harrell,Kevin Durant,Bruno Caboclo,134.826544,32700
4,LaMarcus Aldridge,Montrezl Harrell,Kevin Durant,Miles Bridges,133.960302,32400


### Guards

In [25]:
guards = sgs.assign(foo=1).merge(pgs.assign(foo=1), on='foo').drop('foo', 1)
guards['ffpg'] = guards.ffpg_x + guards.ffpg_y
guards['salary'] = guards.salary_x + guards.salary_y
guards.drop(['ffpg_x', 'salary_x', 'ffpg_y', 'salary_y'], axis=1, inplace=True)

In [26]:
guards.columns = ['sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
guards.shape

(501952, 6)

In [27]:
guards = guards.sort_values(by='points', ascending=False)

In [28]:
gs = guards.iloc[::4, :]

In [29]:
guards.head()

,sg1,sg2,pg1,pg2,points,salary
0,Bradley Beal,DeMar DeRozan,Stephen Curry,Kemba Walker,170.428495,37900
15686,DeMar DeRozan,Bradley Beal,Stephen Curry,Kemba Walker,170.428495,37900
15708,DeMar DeRozan,Bradley Beal,Kemba Walker,Stephen Curry,170.428495,37900
22,Bradley Beal,DeMar DeRozan,Kemba Walker,Stephen Curry,170.428495,37900
15752,DeMar DeRozan,Bradley Beal,Mike Conley,Stephen Curry,166.779585,36700


In [30]:
gs.shape

(125488, 6)

In [31]:
# gs = gs[(gs.sg1 != 'Reggie Bullock') & (gs.sg2 != 'Reggie Bullock')]

### Centers and Forwards

In [32]:
cfs = c.assign(foo=1).merge(forwards.assign(foo=1), on='foo').drop('foo', 1)
cfs['ffpg'] = cfs.FPPG + cfs.ffpg
cfs['salary'] = cfs.Salary + cfs.salary
cfs.drop(['Position', 'FPPG', 'Salary', 'Team', 'Opponent'], axis=1, inplace=True)

In [33]:
cfs.columns = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'points', 'salary']

In [34]:
cfs = cfs.sort_values(by=['points'], ascending=False)

In [35]:
cf = cfs.iloc[::4, :]

In [36]:
cf.head(2)

,c,pf1,pf2,sf1,sf2,points,salary
0,Nikola Jokic,LaMarcus Aldridge,Montrezl Harrell,Kevin Durant,Danilo Gallinari,196.732160,44800
62640,Nikola Jokic,Draymond Green,LaMarcus Aldridge,Kevin Durant,Danilo Gallinari,195.447551,44000


In [37]:
cf.shape

(1338930, 7)

### Full Line

In [38]:
# cols = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
# df = pd.DataFrame(columns=cols)

In [44]:
# keep top 100, if < min of top 100, remove from gs
# take top 100 from line and concat
cols = ['c', 'pf1', 'pf2', 'sf1', 'sf2', 'sg1', 'sg2', 'pg1', 'pg2', 'points', 'salary']
df = pd.DataFrame(columns=cols)
t = time.time()
# gg = gs
for i in range(10001):
    gg = gs
    if df.shape[0] >= 100:
        df = df.sort_values(by=['points'], ascending=False)
        df = df.iloc[:100,:]
        gg =  gg[cf.iloc[i,5]+gg.points > df.points.min()]
       
    cap = gg[cf.iloc[i,6]+gg.salary<=60000]
    if cap.shape[0] > 0:
        cap = cap.sort_values(by=['points'], ascending=False)
        cap = cap.iloc[:100,:]
    else:
        continue
    r = cf.iloc[i,:].to_frame().transpose()
    row = pd.DataFrame(r).assign(foo=1).merge(cap.assign(foo=1), on='foo').drop('foo', 1)
    row['points'] = row.points_x + row.points_y
    row['salary'] = row.salary_x + row.salary_y
    row.drop(['points_x', 'points_y', 'salary_x', 'salary_y'], axis=1, inplace=True)
#     row = pd.concat([cf.iloc[i,:5], 
#            cap['sg1':'pg2'], 
#            pd.Series({'salary': cap.salary+cf.iloc[i,6]}),
#             pd.Series({'points': cap.points+cf.iloc[i,5]})])
    df = df.append(row, ignore_index=True)

df


,c,pf1,pf2,sf1,sf2,sg1,sg2,pg1,pg2,points,salary
0,Nikola Jokic,Marvin Williams,Draymond Green,Kevin Durant,Nicolas Batum,Bogdan Bogdanovic,Jeremy Lamb,Stephen Curry,Patty Mills,288.788,60000
1,Nikola Jokic,Draymond Green,Marvin Williams,Nicolas Batum,Harrison Barnes,Bogdan Bogdanovic,Jeremy Lamb,Stephen Curry,Mike Conley,288.417,59900
2,Nikola Jokic,Marvin Williams,Draymond Green,Kevin Durant,Nicolas Batum,Jeremy Lamb,Bogdan Bogdanovic,Tony Parker,Stephen Curry,287.642,59900
3,Nikola Jokic,JaMychal Green,Marvin Williams,Kevin Durant,Nicolas Batum,Bogdan Bogdanovic,Bradley Beal,Mike Conley,Patty Mills,287.635,60000
4,Nikola Jokic,JaMychal Green,Draymond Green,Kevin Durant,Nicolas Batum,Klay Thompson,Jeremy Lamb,Patty Mills,Mike Conley,287.392,60000
5,Willie Cauley-Stein,Draymond Green,JaMychal Green,Nicolas Batum,Kevin Durant,Jeremy Lamb,Bradley Beal,Patty Mills,Stephen Curry,287.292,60000
6,Nikola Jokic,Marvin Williams,Draymond Green,Kevin Durant,Nicolas Batum,Jeremy Lamb,Bogdan Bogdanovic,Shelvin Mack,Stephen Curry,287.038,59900
7,Nikola Jokic,JaMychal Green,Draymond Green,Kevin Durant,Nicolas Batum,Bogdan Bogdanovic,Jeremy Lamb,Bryn Forbes,Stephen Curry,286.982,59900
8,Nikola Jokic,JaMychal Green,Marvin Williams,Kevin Durant,Nicolas Batum,Jeremy Lamb,Klay Thompson,Bryn Forbes,Stephen Curry,286.97,59900
9,Nikola Jokic,JaMychal Green,Montrezl Harrell,Nicolas Batum,Harrison Barnes,Bogdan Bogdanovic,Jeremy Lamb,Stephen Curry,Mike Conley,286.868,60000


In [ ]:
# set(gs.iloc[1, 0:4].values.tolist()) == set(gs.iloc[2,0:4].values.tolist())

# dups = []
# for i in range(gs.shape[0]-1):
#     check = set(gs.iloc[i, 0:4].values.tolist()) == set(gs.iloc[i+1,0:4].values.tolist())
#     dups.append(check)
# dups = dups + [False]



In [ ]:
# try guards vs cfs instead